In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.utils.data as data
import torchvision
from torchvision import transforms
from torchvision import transforms, datasets, models
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor

In [ ]:
!unzip '/content/sports.zip'

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import os
data_dir = '../content' # directory
print(os.listdir(data_dir))
classes = os.listdir(data_dir + '/train') # train folder grab
print("The length of the folder: ", len(classes)) # find how many folder
print(classes) # disply the folder in a list

['.config', 'drive', 'sample_data']


FileNotFoundError: ignored

In [ ]:

dataset = ImageFolder(data_dir + '/train', transform=ToTensor())
dataset2 = ImageFolder(data_dir + '/valid', transform=ToTensor())

In [ ]:
print(len(dataset.classes))
print(dataset.classes)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.rcParams['figure.facecolor'] = '#ffffff'

In [ ]:
def show_image(img, label):
    print("Label: ", dataset.classes[label], "("+str(label)+")")
    plt.imshow(img.permute(1, 2, 0))
show_image(*dataset[100])

In [ ]:
random_seed = 42
torch.manual_seed(random_seed);
from torch.utils.data.dataset import random_split
val_size = int(len(dataset)*0.2)
train_size = len(dataset) - val_size
train_ds, val_ds = data.random_split(dataset, [train_size, val_size])
len(train_ds), len(val_ds)

In [ ]:
from torch.utils.data.dataloader import DataLoader
batch_size = 50
train_dl = DataLoader(train_ds,
                      batch_size,
                      shuffle=True,
                      num_workers=2,
                      pin_memory= True
                     )
val_dl = DataLoader(val_ds,
                    batch_size*2,
                    num_workers=2,
                    pin_memory=True
                   )

In [ ]:
from torchvision.utils import make_grid

def show_batch_image(dl):
    for images, labels in dl:
        fig, ax = plt.subplots(figsize=(12, 12))
        ax.set_xticks([])
        ax.set_yticks([])
        ax.imshow(make_grid(images, nrow=10).permute(1, 2, 0))
        break
show_batch_image(train_dl)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))
        
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [ ]:
from torchvision.models import resnet152
resnet152 = models.resnet152(pretrained=True)  # deprecated
resnet152


In [ ]:
last_layer = nn.Linear(2048, len(dataset.classes))
resnet152.fc = last_layer
resnet152

In [ ]:
class SportsCnnModel(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        self.network = resnet152
      
    def forward(self, xb):
        return self.network(xb)

In [ ]:
model = SportsCnnModel()
model

In [ ]:
for images, labels in train_dl:
    print('images.shape:', images.shape)
    out = model(images)
    print('out.shape:', out.shape)
    print('out[0]:', out[0])
    break

In [ ]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [ ]:
device = get_default_device()
device

In [ ]:
train_dl = DeviceDataLoader(train_dl, device)
val_dl = DeviceDataLoader(val_dl, device)
to_device(model, device);

In [ ]:
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        model.train()
        train_losses = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
    return history

In [ ]:
model = to_device(SportsCnnModel(), device)

In [ ]:
evaluate(model, val_dl)

In [ ]:
num_epochs = 8   
opt_func = torch.optim.Adam
#lr = 0.001
lr = 0.01

In [ ]:
history = fit(num_epochs, lr, model, train_dl, val_dl, opt_func)

In [ ]:
def plot_accuracies(history):
    accuracies = [x['val_acc'] for x in history]
    plt.plot(accuracies, '-x')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.title('Accuracy vs. No. of epochs');

In [ ]:
plot_accuracies(history)

In [ ]:
def plot_losses(history):
    train_losses = [x.get('train_loss') for x in history]
    val_losses = [x['val_loss'] for x in history]
    plt.plot(train_losses, '-bx')
    plt.plot(val_losses, '-rx')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['Training', 'Validation'])
    plt.title('Loss vs. No. of epochs');
plot_losses(history)

In [ ]:
test_dataset = ImageFolder(data_dir+'/test', transform=ToTensor())

In [ ]:
def predict_image(img, model):
  # convert to a batch of 1
  xb = to_device(img.unsqueeze(0), device)
  # get predictions from model
  yb = model(xb)
  # pic index with highest probability
  _, preds = torch.max(yb, dim=1)
  # retrive the class label
  return dataset.classes[preds[0].item()]

In [ ]:
img, label = test_dataset[0]
plt.imshow(img.permute(1, 2, 0))
print('label:', dataset.classes[label], ', predicted:', predict_image(img, model))

In [ ]:
img, label = test_dataset[111]
plt.imshow(img.permute(1, 2, 0))
print('label:', dataset.classes[label], ', predicted:', predict_image(img, model))

In [ ]:
pip install gluoncv

In [ ]:
#pip install utilies 

In [ ]:
pip install gluoncv

In [ ]:
pip install mxnet 

In [ ]:
import os
import mxnet as mx
from mxnet import gluon, nd, image
from mxnet.gluon.data.vision import transforms
from gluoncv.data.transforms import video
from gluoncv import utils
from gluoncv.model_zoo import get_model
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
from gluoncv.utils import try_import_cv2

transform_fn = transforms.Compose([
    video.VideoCenterCrop(size=224),
    video.VideoToTensor(),
    video.VideoNormalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
def predict_image(img, model):
  # convert to a batch of 1
  xb = to_device(img.unsqueeze(0), device)
  # get predictions from model
  yb = model(xb)
  # pic index with highest probability
  _, preds = torch.max(yb, dim=1)
  # retrive the class label
  return dataset.classes[preds[0].item()]

In [ ]:
transform_fn = transforms.Compose([
    video.VideoCenterCrop(size=224),
    video.VideoToTensor(),
    video.VideoNormalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
model = model.cpu() 

In [ ]:
pip install mxnet 

In [ ]:
import os
import mxnet as mx
from mxnet import gluon, nd, image
from mxnet.gluon.data.vision import transforms
from gluoncv.data.transforms import video
from gluoncv import utils
from gluoncv.model_zoo import get_model
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
from gluoncv.utils import try_import_cv2


object_model = get_model('resnet152_v1', pretrained = True)

In [ ]:
object_model 

In [ ]:
import itertools
from pprint import pprint

co_list = [] 

In [ ]:
pip install tensorflow --upgrade

In [ ]:
tf.executing_eagerly()

In [ ]:
pip install gluoncv 

In [ ]:
from gluoncv.utils import try_import_cv2
from gluoncv import utils
from google.colab.patches import cv2_imshow


cv2 = try_import_cv2()


url_list = []

action_dic = {}  
object_list = {}  


#

for i in range(1,71):
    url = 'https://github.com/suziexi/ucf101/blob/main/temp/' +str(i) + '.mp4?raw=true'+str(i)+'.mp4?raw=true'
    url_list.append(url)

for url in url_list:
  video_fname = utils.download(url) 

  cap = cv2.VideoCapture(video_fname) 
  cnt = 0
  video_frames = []
  while(cap.isOpened()):
      ret, frame = cap.read()
      cnt += 1
      if ret and cnt % 40 == 0:
          video_frames.append(frame)
      if not ret: break
  cap.release()
  act_list = []
  print('We evenly extract %d frames from the video %s.' % (len(video_frames), video_fname))
  if video_frames:
    video_frames_transformed = transform_fn(video_frames)
    final_pred = 0
    obj_final_pred = 0  
     
    for _, frame_img in enumerate(video_frames_transformed):
        yb = model(torch.Tensor(frame_img).unsqueeze(0)) 
        _, pred = torch.max(yb, dim=1)
        obj_pred = object_model(nd.array(frame_img).expand_dims(axis=0))
        obj_final_pred += obj_pred
        if dataset.classes[pred[0].item()] not in act_list: 
          act_list.append(dataset.classes[pred[0].item()])

          if dataset.classes[pred[0].item()] not in act_dic: 
            act_dic[dataset.classes[pred[0].item()]] = 1 
          else: 
            act_dic[dataset.classes[pred[0].item()]] += 1 

    print(act_list) 
    

    obj_final_pred /= len(video_frames)

    classes = object_model.classes 
    topK = 4
    ind = nd.topk(obj_final_pred, k=topK)[0].astype('int')
    
    this_frame = []  
    print('[Object Detection] The input video is classified to be')
    for i in range(topK):
        print('\t[%s], with  probability %.3f.'%
              (classes[ind[i].asscalar()], nd.softmax(obj_final_pred)[0][ind[i]].asscalar()))
        if classes[ind[i].asscalar()] in object_list: 
          object_list[classes[ind[i].asscalar()] ] += nd.softmax(obj_final_pred)[0][ind[i]].asscalar()
          this_frame.append(classes[ind[i].asscalar()])
          
        else: 
          object_list[classes[ind[i].asscalar()] ] = nd.softmax(obj_final_pred)[0][ind[i]].asscalar() 
          this_frame.append(classes[ind[i].asscalar()])
        


print(act_dic)

In [ ]:
# Code reference: https://www.kaggle.com/datasets/gpiosenka/sports-classification
# https://www.kaggle.com/code/mahfuzraihan/sportsclassification-resnet50 